# Scrape and Save
This notebook was written to automate the scraping and saving of images from the internet.

Import the relevant modules.

In [ ]:
pip install webdriver-manager

In [ ]:
import requests
from bs4 import BeautifulSoup
import os
from selenium import webdriver 
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

The next step is to get the html as text from the url.

In [ ]:
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))

url = "https://www.thecarconnection.com/cars/acura_ilx"
remove = "https://images.hgmsites.net/tmb/"
driver.get(url)

Next, we use BeautifulSoup to parse the html text data for the image links and then save them in a list.

In [ ]:
driver.execute_script("window.scrollTo(0, 0);")

html = driver.page_source
soup = BeautifulSoup(html, 'html.parser')
containers = soup.findAll('div', {'class':"isv-r PNCib MSM1fd BUooTd"} )

os is used to a make the folder into which the files will be saved. A list of directory names corresponding to the category of images should be created.

In [ ]:
os.getcwd()

In [ ]:
dirs = ['c:\\Users\\ashle\\OneDrive\\Documents\\GitHub\\image_saver\\images2']
image_elements = []

In [ ]:
# find the "Makes" dropdown and click it, get a list and iterate through each of them
makes_dropdown = WebDriverWait(driver, 60).until(
    EC.presence_of_element_located((By.XPATH, """//*[@id="save-new-car"]/span[1]/span"""))
)
makes_dropdown.click()

makes = makes_dropdown.find_elements(By.XPATH, """//*[@id="save-new-car"]/span[1]/span""")
for make in makes:
    make.click()

In [ ]:
# find the "Models" dropdown and click it, get a list and iterate through each of them
models_dropdown = WebDriverWait(driver, 60).until(
        EC.presence_of_element_located((By.XPATH, """//*[@id="save-new-car"]/span[2]/span"""))
    )
models_dropdown.click()

models = models_dropdown.find_elements(By.TAG_NAME, """//*[@id="save-new-car"]/span[2]/span""")
for model in models:
    model.click()

In [ ]:
 # find the "Years" dropdown and click it, get a list and iterate through each of them
years_dropdown = WebDriverWait(driver, 60).until(
            EC.presence_of_element_located((By.XPATH, """//*[@id="save-new-car"]/span[3]/span"""))
        )
years_dropdown.click()
        
years = years_dropdown.find_elements(By.TAG_NAME, """//*[@id="save-new-car"]/span[3]/span""")
for year in years:
    year.click()
            

In [ ]:
#GO
go_button = WebDriverWait(driver, 60).until(
            EC.presence_of_element_located((By.XPATH, """//*[@id="research-dd"]/div/a"""))
        )
go_button.click()

Finally, iterating through the list of image urls, download and save the images in the appropriate directory. 

Images on page are: thumb (tmb), small, medium, large and huge. --> There is only one clear large image per page 

**Issue: only downloading one image per page - need to navigate through each page using sel commands BUT still not sure how to exclude images showing the interior or accessories**

Another option would be to chnage size of small images , but will lose definition in doing so...

In [ ]:
image_elements_with_tmb = soup.find('div', {'class':"image"})
print(image_elements_with_tmb['data-image-large'])
image_elements.append(image_elements_with_tmb)
# for img in image_elements_with_tmb:

#     if img.has_attr('data-image-large'): #and not img['src'].startswith(remove):
#         # print(img['data-image-large'])
#         image_elements.append(img)
        

In [ ]:
img_urls = []
for element in image_elements:
     img_urls.append(element['src'])  

In [ ]:
for i, url in enumerate(img_urls):
  try: response = requests.get(url)
  except: pass
  
  if response.status_code:
    with open('{0}/test_save_{1}.png'.format(dirs[0], i), 'wb') as fp:
          fp.write(response.content)

# Saving to Google Cloud

1. Install gcloud https://cloud.google.com/sdk/docs/install#installation_instructions. 

2. Follow the instructions here https://cloud.google.com/docs/authentication/provide-credentials-adc#local-dev. 

3. Follow the instructions at https://cloud.google.com/docs/authentication/client-libraries to set up Google API credentials.

In [ ]:
from google.cloud import storage
import glob

project_id = 'vehicle-recognition-model'

storage_client = storage.Client(project=project_id)

bucket_name = 'vehicle-recognition-model'

bucket = storage_client.get_bucket(bucket_name)

In [ ]:
for str_path_file in glob.glob(dirs[0] + '/**'):
    blob = bucket.blob(str_path_file)
    blob.upload_from_filename(str_path_file)

We'll need to change this script in the following ways


* It will need to be able to download text from an accessible website or google images; https://www.thecarconnection.com/ is not accessible via this method.
* Depending on the website, the script may need to be able to perform searches 
* We will need to direct it to save the images in a Google Cloud bucket directory.
* Filter out unneccsary images
* How much is too much to download at a time
* Upload directly to cloud bucket
